## Modelo 3 - Bert

*   Arquitectura: Transformer
*   Fine-Tunning: Letras de canciones que se consideran aptas o no
*   Repositorio: https://colab.research.google.com/drive/1QQ5N1v5FLoXFXMeYHB8wVzvyxNT9X5oA?usp=sharing#scrollTo=t-OS1v4fbiYh

*   https://github.com/dccuchile/beto?tab=readme-ov-file
*   Modelo Específico: Bert - BETO

In [ ]:
import torch
from torch import nn, optim
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from textwrap import wrap

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# path_checkpoint = "Checkpoint/modelo_BETO_ultimo_checkpoint.pth"
path_checkpoint = "Checkpoint/modelo_BETO_ultimo_checkpoint_state_dict.pth"

category_mapping = {
    '0': 0,
    '1': 1,
}

In [ ]:
PRE_TRAINED_MODEL_NAME = 'ignacio-ave/BETO-nlp-sentiment-analysis-spanish'

In [ ]:
class BETOMusicForKidsClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BETOMusicForKidsClassifier, self).__init__()

    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    outputs = self.bert(
                        input_ids = input_ids,
                        attention_mask = attention_mask
                        )
    pooled_output = outputs.pooler_output
    drop_output = self.drop(pooled_output)
    output = self.linear(drop_output)
    return output

In [ ]:
# Inicializa el modelo
n_classes = len(category_mapping)
model_cargado = BETOMusicForKidsClassifier(n_classes)

# Cargar el state_dict
checkpoint = torch.load(path_checkpoint, map_location=torch.device('cpu'))
model_cargado.load_state_dict(checkpoint)

# Mueve el modelo al dispositivo adecuado
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_cargado.to(device)

BETOMusicForKidsClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## Predicción

In [ ]:
path_voice_to_text = 'Separado/Calle 13 - Latinoamérica (Letra).txt'

In [ ]:
def predict_genre(text, model, tokenizer, device, max_length=200):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(outputs, dim=1)

    # Mapear el número de clase predicho al nombre de la categoría correspondiente
    predicted_genre = list(category_mapping.keys())[list(category_mapping.values()).index(predicted_class.item())]
    return predicted_genre

In [ ]:
# PRE_TRAINED_MODEL_NAME = 'ignacio-ave/BETO-nlp-sentiment-analysis-spanish'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
# Lee el contenido del archivo .txt
with open(path_voice_to_text, 'r', encoding='utf-8') as file:
    transcribed_text = file.read()

# Imprime el contenido del archivo
print(transcribed_text)

# Realiza la predicción de género utilizando el texto transcribido
genre = predict_genre(transcribed_text, model_cargado, tokenizer, device)
print(f"\n Predicción (0 = No Apta, 1 = Apta): {genre}")


 que me enseñó mi padre el que no quiere a su patria no quiere a su madre soy américa latina un pueblo sin pierna pero que camina oye tú no puedes comprar el viento tú no puedes comprar el sol tú no puedes comprar la lluvia tú no puedes comprar el calor tú no puedes comprar las nubes tú no puedes comprar los colores tú no puedes comprar mi alegría tú no puedes comprar

 Predicción (0 = No Apta, 1 = Apta): 1


C:\Users\LENOVO LOQ\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


---
---
---

In [ ]:
# Pollitos

test_text = "Los pollitos dicen droga, droga, droga. Cuando tienen hambre Cuando tienen frío"
genre = predict_genre(test_text, model_cargado, tokenizer, device)
print("\n".join(wrap(test_text)))
print(f"\nPredicción (0 = No Apta, 1 = Apta): {genre}")

Los pollitos dicen droga, droga, droga. Cuando tienen hambre Cuando
tienen frío

Predicción (0 = No Apta, 1 = Apta): 0


---


---



---




---



---



---



---



---



---

### FLASK

In [ ]:
!pip install flask

In [ ]:
!pip install torch --user


  Using cached torch-2.3.0-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached typing_extensions-4.12.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.3.0-cp312-cp312-win_amd64.whl (159.7 MB)
Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
Using cached typing_extensions-4.12.0-py3-none-any.whl (37 kB)
Using cached filelock-3.14.0-py3-none-any.whl (12 kB)
Using cached fsspec-2024.5.0-py3-none-any.whl (316 kB)
Using cached n

In [ ]:
!pip install transformers

  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.23.1-py3-none-any.whl.metadata (12 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.5.15-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.2-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.4.3-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
Using cached transformers-4.41.1-py3-none-any.whl (9.1 MB)
Using cached huggingface_hub-0.23.1-py3-none-any.whl (401 kB)
Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl (138 kB)
Using cached regex-2024.5.15-cp312-cp312-win_amd64.whl (268 kB)
Using cached safetensors-0.4.3-cp312-non

ERROR: Could not install packages due to an OSError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'C:\\Python312\\Scripts\\normalizer.exe' -> 'C:\\Python312\\Scripts\\normalizer.exe.deleteme'



In [ ]:
from flask import Flask, request, jsonify
import torch
from torch import nn
from transformers import BertModel, BertTokenizer
import os

In [ ]:
app = Flask(__name__)

In [ ]:
# Configuración del modelo y el tokenizador
PRE_TRAINED_MODEL_NAME = 'ignacio-ave/BETO-nlp-sentiment-analysis-spanish'
path_checkpoint = "/content/drive/MyDrive/Proyecto de IA/Checkpoints/modelo_BETO_ultimo_checkpoint.pth"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

category_mapping = {
    '0': 0,
    '1': 1,
}

In [ ]:
class BETOMusicForKidsClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BETOMusicForKidsClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
                            input_ids=input_ids,
                            attention_mask=attention_mask
                            )
        pooled_output = outputs.pooler_output
        drop_output = self.drop(pooled_output)
        output = self.linear(drop_output)
        return output

In [ ]:
# Cargar el modelo y el tokenizador
model = BETOMusicForKidsClassifier(n_classes=2)
model = torch.load(path_checkpoint)
# model.load_state_dict(torch.load(path_checkpoint, map_location=device))
model = model.to(device)
model.eval()

In [ ]:

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
def predict_genre(text, model, tokenizer, device, max_length=200):
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(outputs, dim=1)

    predicted_genre = list(category_mapping.keys())[list(category_mapping.values()).index(predicted_class.item())]
    return predicted_genre

In [ ]:

@app.route('/classify', methods=['POST'])
def classify():
    data = request.json
    if 'text' not in data:
        return jsonify({'error': 'No text provided'}), 400
    
    text = data['text']
    genre = predict_genre(text, model, tokenizer, device)
    return jsonify({'genre': genre})

if __name__ == '__main__':
    app.run(debug=True)